# Relatório de Resultados das Operações

## Importando as bibliotecas e carregando os arquivos necessários

In [1]:
#Importa as bibliotecas necessárias
import pandas as pd
import numpy as np
import datetime
from plotly.offline import iplot
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import os

In [2]:
#Cria as pastas necessárias
pasta ="./relatorio/pags"

access_rights = 0o777

try:
    os.mkdir('relatorio', access_rights)
except OSError:
    print ("\tAVISO: A criação da pasta (relatorio) falhou ou a pasta já existe")
else:
    print ("\tPasta (relatorio) criada com sucesso!")

try:
    os.mkdir('relatorio/pags', access_rights)
except OSError:
    print ("\tAVISO: A criação da pasta (pags) falhou ou a pasta já existe")
else:
    print ("\tPasta (pags) criada com sucesso!")

	AVISO: A criação da pasta (relatorio) falhou ou a pasta já existe
	AVISO: A criação da pasta (pags) falhou ou a pasta já existe


In [3]:
#Define as funções personalizadas necessárias
def write_to_html_file(df, title='', filename='out.html'):
    result = '''
<html>
<head>
<style>

    h2 {
        text-align: center;
        font-family: Helvetica, Arial, sans-serif;
    }
    body{ 
        display: flex;
        align-items: center;
        justify-content: center;
    }
    table { 
        margin-left: auto;
        margin-right: auto;
        width: 90%;
        color: #707070;
    }
    table, th, td {
        border: 1px solid #707070;
        border-collapse: collapse;
    }
    th {
        padding: 5px;
        text-align: left;
        font-family: Helvetica, Arial, sans-serif;
        font-size: 90%;
    }
    td {
        padding: 5px;
        text-align: right;
        font-family: Helvetica, Arial, sans-serif;
        font-size: 90%;
    }
    table tbody tr:hover {
        background-color: #dddddd;
    }
    .wide {
        width: 90%; 
    }

</style>
</head>
<body>
    '''
    result += '<h2> %s </h2>\n' % title
    if type(df) == pd.io.formats.style.Styler:
        result += df.render()
    else:
        result += df.to_html(classes='wide', escape=False)
    result += '''
</body>
</html>
'''
    with open(filename, 'w') as f:
        f.write(result)

def highlight_max(s):    
    is_max = s == s.max()
    return ['color: white; background-color: #3749E9' if v else '' for v in is_max]

def highlight_min(s):
    is_max = s == s.min()
    return ['color: white; background-color: #112244' if v else '' for v in is_max]

def destaque(val):
    color = '#F2F200' if val == 'Sim' else ''
    return 'background: {}'.format(color)

def destaque_coluna(val):
    color = '#F2F200' if val == 0 else ''
    return 'background: {}'.format(color)

In [4]:
# DataFrame principal.
dados_STG_OPR_ITT = pd.read_excel('STG_OPR_ITT.xlsx') # DataFrame de Operações.

# DataFrame para validação (esse será usado para validar as modalidades do principal).
dados_STG_MDL = pd.read_excel('STG_MDL.xlsx') # DataFrame das Modalidades.

In [5]:
#dados_STG_OPR_ITT

## Filtrando os casos especiais e analisando o preenchimento das células

In [6]:
# Tratando as datas.
dados_STG_OPR_ITT.DAT_RSS_FNT_ITT = pd.to_datetime(dados_STG_OPR_ITT.DAT_RSS_FNT_ITT, format='%Y-%m-%d', errors='coerce')
dados_STG_OPR_ITT.DAT_INC_DBO = pd.to_datetime(dados_STG_OPR_ITT.DAT_INC_DBO, format='%Y-%m-%d', errors='coerce')

# Completando espaços vázios com zeros.
dados_STG_OPR_ITT = dados_STG_OPR_ITT.fillna(0)

dict_tipo = {'ID_STG_OPR_ITT': np.int64, 'VLR_CTRD_CSC': np.float64, 'QTD_PCL': np.int64, 'VLR_SDO_DDR': np.float64,
             'QTD_CLI_CAD_POS': np.int64, 'QTD_OPR': np.int64, 'ID_FNT_ITT': np.int64, 'ID_MDL': str,
             'DES_TIP_PSS': str}

for coluna, tipo in dict_tipo.items():
    dados_STG_OPR_ITT[coluna] = dados_STG_OPR_ITT[coluna].astype(tipo)
    
#dados_STG_OPR_ITT.dtypes

## Convertendo os valores para acrescentar dois decimais

In [7]:
# Colunas a serem convertidas.
colunas = ['VLR_CTRD_CSC', 'VLR_SDO_DDR']

for coluna in colunas:
#    print('-' * 60 + f'\nColuna: {coluna}\n')
    
    for index, antes in enumerate(dados_STG_OPR_ITT[coluna].dropna()):
        depois = antes * 0.01
        dados_STG_OPR_ITT[coluna] = dados_STG_OPR_ITT[coluna].replace(antes, depois) # Trocando o velho pelo novo.

#        if index < 10: # Essa parte serve somente para limitar a amostragem dos valores.
#            print(f'R${depois:>20.2f}') # Mostrando alguns exemplos.
    
#    print('-' * 60)

## Adicionando uma nova coluna (MDL_DESCRICAO) ao DataFrame principal

### Primeiramente criando um dicionário com o código e sua respectiva descrição para facilitar o processo

In [8]:
# Escolhendo as colunas necessárias.
new = dados_STG_MDL[['COD_MDL', 'DES_MDL']] 
dictionary = new.set_index('COD_MDL').T.to_dict('list') # Convertendo em um dicionário (valores em lista).

for key, value in dictionary.items():
    dictionary[key] = str(value).strip("['']") # Convertendo os valores (em lista) para string.
    
#dictionary # Mostrando o dicionário final.

### Linha por linha atribuindo ao respectivo código a sua descrição

In [9]:
# Linha por linha atribuindo ao respectivo código a sua descrição
x = [dictionary.get(codigo, f'{codigo} (MODALIDADE NAO ENCONTRADA)') 
     for index, codigo in enumerate(dados_STG_OPR_ITT['ID_MDL'])]

dados_STG_OPR_ITT['MDL_DESCRICAO'] = x

#dados_STG_OPR_ITT[['ID_MDL', 'MDL_DESCRICAO']].head()

## Analisando a soma das operações por modalidades

In [10]:
#Cria um dataframe
analisar = dados_STG_OPR_ITT[['MDL_DESCRICAO', 'QTD_OPR']].dropna()

operacoes = analisar.groupby('MDL_DESCRICAO').sum().sort_values(by = 'QTD_OPR', ascending = False)
#operacoes

### Criando um gráfico

In [11]:
#Define escalas para o tamanho máximo padrão dos gráficos
operacoes_range = [0, round(max(operacoes.QTD_OPR))]

#Filtra valores menores ou iguais a 1 milhão
operacoes_menor = operacoes[operacoes['QTD_OPR'] > 0]
operacoes_menor = operacoes_menor[operacoes_menor['QTD_OPR'] <= 1000]
operacoes_menor = operacoes_menor.sort_values(by = 'QTD_OPR', ascending = True)

#Filtra valores menores ou iguais a 1 milhão
operacoes_medio = operacoes[operacoes['QTD_OPR'] > 1000]
operacoes_medio = operacoes_medio[operacoes_medio['QTD_OPR'] <= 50000]
operacoes_medio = operacoes_medio.sort_values(by = 'QTD_OPR', ascending = True)

#Filtra valores maiores que 1 milhão e menores e iguais a 5 bilhões
operacoes_maior = operacoes[operacoes['QTD_OPR'] > 50000]
operacoes_maior = operacoes_maior.sort_values(by = 'QTD_OPR', ascending = True)

fig = make_subplots(rows=3, cols=1, row_heights=[0.3, 0.6, 0.4])

fig.add_trace(go.Bar(y=operacoes_menor.index,
                     x=operacoes_menor['QTD_OPR'],
                     orientation = 'h',
                     marker=dict(color='#112244')
                    ),
              row = 3, col = 1
)

fig.add_trace(go.Bar(y = operacoes_medio.index,
                     x = operacoes_medio['QTD_OPR'],
                     orientation = 'h',
                     marker = dict(color = '#112244')
                    ),
              row = 2, col = 1
)

fig.add_trace(go.Bar(y = operacoes_maior.index,
                     x = operacoes_maior['QTD_OPR'],
                     orientation = 'h',
                     marker = dict(color = '#112244')
                    ),
              row = 1, col = 1
)

fig.update_yaxes(showline=True, linewidth=1, linecolor='#717171')
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='#D9D9DE')
fig.update_xaxes(range=operacoes_range)
fig.update_layout(dict(plot_bgcolor = '#FFFFFF', paper_bgcolor = '#FFFFFF'))
fig.update_layout(showlegend=False)

iplot(fig)

fig.write_html(pasta + "/opr_soma_opr_por_mdl.html")

## Analisando a soma das parcelas por modalidades

In [12]:
#Cria um dataframe
analisar = dados_STG_OPR_ITT[['MDL_DESCRICAO', 'QTD_PCL']].dropna()

parcelas = analisar.groupby('MDL_DESCRICAO').sum().sort_values(by = 'QTD_PCL', ascending = False)
#parcelas

### Criando um gráfico

In [13]:
#Define escalas para o tamanho máximo padrão dos gráficos
parcelas_range = [0, round(max(parcelas.QTD_PCL))]

#Filtra valores menores ou iguais a 1 milhão
parcelas_menor = parcelas[parcelas['QTD_PCL'] > 0]
parcelas_menor = parcelas_menor[parcelas_menor['QTD_PCL'] <= 500]
parcelas_menor = parcelas_menor.sort_values(by = 'QTD_PCL', ascending = True)

#Filtra valores menores ou iguais a 1 milhão
parcelas_medio = parcelas[parcelas['QTD_PCL'] > 500]
parcelas_medio = parcelas_medio[parcelas_medio['QTD_PCL'] <= 5000]
parcelas_medio = parcelas_medio.sort_values(by = 'QTD_PCL', ascending = True)

#Filtra valores maiores que 1 milhão e menores e iguais a 5 bilhões
parcelas_maior = parcelas[parcelas['QTD_PCL'] > 5000]
parcelas_maior = parcelas_maior.sort_values(by = 'QTD_PCL', ascending = True)

fig = make_subplots(rows=1, cols=3, column_widths=[0.6, 0.2, 0.5])

fig.add_trace(go.Bar(x=parcelas_menor.index,
                     y=parcelas_menor['QTD_PCL'],
                     marker=dict(color='#3749E9')
                    ),
              row = 1, col = 1
)

fig.add_trace(go.Bar(x = parcelas_medio.index,
                     y = parcelas_medio['QTD_PCL'],
                     marker = dict(color = '#3749E9')
                    ),
              row = 1, col = 2
)

fig.add_trace(go.Bar(x = parcelas_maior.index,
                     y = parcelas_maior['QTD_PCL'],
                     marker = dict(color = '#3749E9')
                    ),
              row = 1, col = 3
)

fig.update_xaxes(showline=True, linewidth=1, linecolor='#717171')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#D9D9DE')
fig.update_yaxes(range=parcelas_range)
fig.update_layout(dict(plot_bgcolor = '#FFFFFF', paper_bgcolor = '#FFFFFF'))
fig.update_layout(showlegend=False)

iplot(fig)

fig.write_html(pasta + "/opr_soma_pcl_por_mdl.html")

## Analisando a soma de clientes por modalidades

In [14]:
#Cria um dataframe
analisar = dados_STG_OPR_ITT[['MDL_DESCRICAO', 'QTD_CLI_CAD_POS']].dropna()

clientes = analisar.groupby('MDL_DESCRICAO').sum().sort_values(by = 'QTD_CLI_CAD_POS', ascending = False)
#clientes

### Criando um gráfico

In [15]:
#Define escalas para o tamanho máximo padrão dos gráficos
clientes_range = [0, round(max(clientes.QTD_CLI_CAD_POS))]

#Filtra valores menores ou iguais a 1 milhão
clientes_menor = clientes[clientes['QTD_CLI_CAD_POS'] > 0]
clientes_menor = clientes_menor[clientes_menor['QTD_CLI_CAD_POS'] <= 1000]
clientes_menor = clientes_menor.sort_values(by = 'QTD_CLI_CAD_POS', ascending = True)

#Filtra valores menores ou iguais a 1 milhão
clientes_medio = clientes[clientes['QTD_CLI_CAD_POS'] > 1000]
clientes_medio = clientes_medio[clientes_medio['QTD_CLI_CAD_POS'] <= 100000]
clientes_medio = clientes_medio.sort_values(by = 'QTD_CLI_CAD_POS', ascending = True)

#Filtra valores maiores que 1 milhão e menores e iguais a 5 bilhões
clientes_maior = clientes[clientes['QTD_CLI_CAD_POS'] > 100000]
clientes_maior = clientes_maior.sort_values(by = 'QTD_CLI_CAD_POS', ascending = True)

fig = make_subplots(rows=1, cols=3, column_widths=[0.5, 0.5, 0.3])

fig.add_trace(go.Bar(x=clientes_menor.index,
                     y=clientes_menor['QTD_CLI_CAD_POS'],
                     marker=dict(color='#112244')
                    ),
              row = 1, col = 1
)

fig.add_trace(go.Bar(x = clientes_medio.index,
                     y = clientes_medio['QTD_CLI_CAD_POS'],
                     marker = dict(color = '#112244')
                    ),
              row = 1, col = 2
)

fig.add_trace(go.Bar(x = clientes_maior.index,
                     y = clientes_maior['QTD_CLI_CAD_POS'],
                     marker = dict(color = '#112244')
                    ),
              row = 1, col = 3
)

fig.update_xaxes(showline=True, linewidth=1, linecolor='#717171')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#D9D9DE')
fig.update_yaxes(range=clientes_range)
fig.update_layout(dict(plot_bgcolor = '#FFFFFF', paper_bgcolor = '#FFFFFF'))
fig.update_layout(showlegend=False)

iplot(fig)

fig.write_html(pasta + "/opr_soma_cliente_por_mdl.html")

## Agrupando clientes e parcelas por modalidades

In [16]:
#Plota em formato tabela e exporta para HTML
data = [go.Bar(x=clientes.index,
               y=clientes.QTD_CLI_CAD_POS,
               name='Número de Clientes',
               marker=dict(color='#3749E9')
              ),
        go.Bar(x=parcelas.index,
               y=parcelas.QTD_PCL,
               name='Número de Parcelas',
               marker=dict(color='#112244')
              )]

fig = go.Figure(data=data)

fig = go.Figure(data=data)
fig.update_xaxes(showline=True, linewidth=1, linecolor='#717171')
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='#D9D9DE')
fig.update_layout(dict(plot_bgcolor = '#FFFFFF', paper_bgcolor = '#FFFFFF'))
fig.update_layout(xaxis={'categoryorder':'total ascending'}, showlegend=True)

iplot(fig)
fig.write_html(pasta + "/opr_clientes_parcelas.html")

## Adicionando uma nova coluna (VLR_PAGO) ao DataFrame temporário

### Primeiramente criando uma lista com o resultado da subtração

In [17]:
# Selecionando somente as linhas com o ID_MDL igual a C01 (consórcio).
analisar = dados_STG_OPR_ITT[dados_STG_OPR_ITT['ID_MDL'] == 'C01'] 
analisar = analisar[['VLR_CTRD_CSC', 'VLR_SDO_DDR']] # Selecionando somente as colunas necessárias.

total = [] # Lista temporária.

for index in analisar.index: # Efeituando a conta e jogando o resultado na lista.
    total.append(round(analisar['VLR_CTRD_CSC'][index] - analisar['VLR_SDO_DDR'][index], 2))

    
analisar['VLR_PAGO'] = total # Adicionando a lista.
#analisar.head(10) 

In [18]:
#Plota em formato grafico
lista_somas = [analisar['VLR_SDO_DDR'].sum(), analisar['VLR_CTRD_CSC'].sum(), analisar['VLR_PAGO'].sum()]
lista_nomes = ['Saldo Devedor', 'Valor Contratado', 'Valor Pago']

data = [go.Bar(x = lista_nomes,
               y = lista_somas,
               marker = dict(color = ['#FFAA00', '#112244', '#00B7CC'])
              )]

fig = go.Figure(data = data)
fig.update_xaxes(showline = True, linewidth = 1, linecolor = '#717171')
fig.update_yaxes(showgrid = True, gridwidth = 1, gridcolor = '#D9D9DE')
fig.update_layout(dict(plot_bgcolor = '#FFFFFF', paper_bgcolor = '#FFFFFF'))
fig.update_layout(showlegend = False)

iplot(fig)
fig.write_html(pasta + "/opr_grafico_valor_consorcio.html")

## Dataframe resumido

In [19]:
#Cria uma nova tabela com os dados resumidos
tabela = dados_STG_OPR_ITT[['MDL_DESCRICAO', 'QTD_PCL', 'QTD_CLI_CAD_POS', 'QTD_OPR']]
tabela = tabela.groupby('MDL_DESCRICAO').sum()
tabela = tabela.rename(columns={'QTD_PCL': 'Número de Parcelas', 'QTD_CLI_CAD_POS': 'Número de Clientes',
                                'QTD_OPR': 'Número de Operações'}).sort_values(by = 'Número de Parcelas', ascending = True)
tabela = tabela.rename_axis('Modalidades', axis='columns')
tabela.index.name = None

opr_por_cli = [(tabela['Número de Operações'][x] / tabela['Número de Clientes'][x]) for x in range (len (tabela))]
tabela['Número de Operações por Cliente'] = opr_por_cli

visualiza = tabela.replace(to_replace = 0, value = np.nan)

visualiza = (visualiza
                .style
                .format({'Número de Operações por Cliente':"{:,.2f}"}, na_rep="-")
                .highlight_null(null_color='#F2F200')
                .apply(highlight_max)
                .apply(highlight_min))
write_to_html_file(visualiza, title='', filename=pasta + "/opr_tab_modalidades.html")
visualiza

## Calculando os indicadores gerais

In [20]:
#Calcula os resultados totais da remessa
total_vlr_contratado = dados_STG_OPR_ITT['VLR_CTRD_CSC'].sum()
total_qtd_pcl = dados_STG_OPR_ITT['QTD_PCL'].sum()
total_vlr_saldo_devedor = dados_STG_OPR_ITT['VLR_SDO_DDR'].sum()
total_qtd_opr = dados_STG_OPR_ITT['QTD_OPR'].sum()
total_opr_por_cli = tabela['Número de Operações'].sum() / tabela['Número de Clientes'].sum()

#total_vlr_contratado, total_qtd_pcl, total_vlr_saldo_devedor, total_qtd_opr, total_opr_por_cli